In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
import nasapy
import nasa



from api_keys import api_key


In [4]:
Asteroids = pd.read_csv("Asteroid.csv")
Asteroids

,Date,ID,Name,Absolute Magnitude,Estimated Max Diameter,Estimated Min Diameter,Potentially Hazardous?,Miss Distance (km),Orbiting Body
0,2019-01-01,3837557,(2019 AN),24.200,0.085909,0.038420,False,1.910837e+07,Earth
1,2019-01-01,3398654,(2007 YS56),25.700,0.043057,0.019256,False,1.798761e+07,Earth
2,2019-01-01,3398652,(2007 YQ56),19.900,0.622358,0.278327,True,1.144099e+07,Earth
3,2019-01-01,3837538,(2018 YV2),24.200,0.085909,0.038420,False,1.022072e+07,Earth
4,2019-01-01,3771017,(2017 EV2),20.800,0.411188,0.183889,False,7.033567e+07,Earth
...,...,...,...,...,...,...,...,...,...
5044,2019-12-31,2006239,6239 Minos (1989 QF),18.500,1.185878,0.530341,True,6.346829e+07,Earth
5045,2019-12-31,3555072,(2011 AH5),26.000,0.037501,0.016771,False,4.341405e+07,Earth
5046,2019-12-31,3976544,(2020 AA2),27.000,0.023661,0.010582,False,1.643657e+06,Earth
5047,2019-12-31,3893722,(2019 WR4),26.065,0.036395,0.016276,False,4.493435e+06,Earth


In [6]:
Asteroids.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5049 entries, 0 to 5048
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Date                    5049 non-null   object 
 1   ID                      5049 non-null   int64  
 2   Name                    5049 non-null   object 
 3   Absolute Magnitude      5049 non-null   float64
 4   Estimated Max Diameter  5049 non-null   float64
 5   Estimated Min Diameter  5049 non-null   float64
 6   Potentially Hazardous?  5049 non-null   bool   
 7   Miss Distance (km)      5049 non-null   float64
 8   Orbiting Body           5049 non-null   object 
dtypes: bool(1), float64(4), int64(1), object(3)
memory usage: 320.6+ KB


In [7]:
Asteroids["ID"].nunique()

4510

In [11]:
Asteroids_by_ID=Asteroids.groupby('ID')
Asteroids_by_ID.head()

,Date,ID,Name,Absolute Magnitude,Estimated Max Diameter,Estimated Min Diameter,Potentially Hazardous?,Miss Distance (km),Orbiting Body
0,2019-01-01,3837557,(2019 AN),24.200,0.085909,0.038420,False,1.910837e+07,Earth
1,2019-01-01,3398654,(2007 YS56),25.700,0.043057,0.019256,False,1.798761e+07,Earth
2,2019-01-01,3398652,(2007 YQ56),19.900,0.622358,0.278327,True,1.144099e+07,Earth
3,2019-01-01,3837538,(2018 YV2),24.200,0.085909,0.038420,False,1.022072e+07,Earth
4,2019-01-01,3771017,(2017 EV2),20.800,0.411188,0.183889,False,7.033567e+07,Earth
...,...,...,...,...,...,...,...,...,...
5044,2019-12-31,2006239,6239 Minos (1989 QF),18.500,1.185878,0.530341,True,6.346829e+07,Earth
5045,2019-12-31,3555072,(2011 AH5),26.000,0.037501,0.016771,False,4.341405e+07,Earth
5046,2019-12-31,3976544,(2020 AA2),27.000,0.023661,0.010582,False,1.643657e+06,Earth
5047,2019-12-31,3893722,(2019 WR4),26.065,0.036395,0.016276,False,4.493435e+06,Earth


In [ ]:
Asteroid_name=Asteroids_by_ID["ID"]


plt.bar